# HW7

In [12]:
import matplotlib
import numpy as np
from numpy import linalg as LA
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import math
import scipy as sp
from scipy import io
from scipy.stats import logistic as sig
import pandas as pd

np.set_printoptions(threshold=np.nan)

import sklearn
from sklearn.preprocessing import normalize
import csv
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle as shuffle
from sklearn.utils import resample as resample
from sklearn.preprocessing import StandardScaler
%matplotlib inline

# NP FNS

In [13]:
def vec(arr): # d b 1 --> (d, 1)
    return arr.reshape((arr.shape[0], 1))

In [14]:
def arr(vec): # 1 by d --> (d, )
    return vec.flatten()

# Problem 1: Lloyd's Method on MNIST

In [15]:
import math
import random

In [16]:
dict_mnist_data = sp.io.loadmat("mnist_data/images.mat")
points = dict_mnist_data['images']
points.T.shape

(60000, 28, 28)

In [176]:
# class Point(object):
#     '''
#     A point in n dimensional space
#     '''
#     def __init__(self, coords):
#         '''
#         coords - A list of values, one per dimension
#         '''

#         self.coords = coords
#         self.n = len(coords)

#     def __repr__(self):
#         return str(self.coords)


# class Cluster:
#     '''https://gist.github.com/iandanforth/5862470'''
#     def __init__(points):
#         if len(points) == 0:
#             raise Exception("ERROR: empty cluster")
#         self.points = points
        
#         # The dimensionality of the points in this cluster
#         self.n = points[0].n

#         # Assert that all points are of the same dimensionality
#         for p in points:
#             if p.n != self.n:
#                 raise Exception("ERROR: inconsistent dimensions")
        
#         # Set up the initial centroid (this is usually based off one point)
#         self.centroid = self.calculateCentroid()

#     def __repr__(self):
#         '''
#         String representation of this object
#         '''
#         return str(self.points)
    
    
#     def update(self, points):
#         '''
#         Returns the distance between the previous centroid and the new after
#         recalculating and storing the new centroid.
#         Note: Initially we expect centroids to shift around a lot and then
#         gradually settle down.
#         '''
#         old_centroid = self.centroid
#         self.points = points
#         self.centroid = self.calculateCentroid()
#         shift = getDistance(old_centroid, self.centroid)
#         return shift
    
#     def calculateCentroid(self):
#         '''
#         Finds a virtual center point for a group of n-dimensional points
#         '''
#         numPoints = len(self.points)
#         # Get a list of all coordinates in this cluster
#         coords = [p.coords for p in self.points]
#         # Reformat that so all x's are together, all y'z etc.
#         unzipped = zip(*coords)
#         # Calculate the mean for each dimension
#         centroid_coords = [math.fsum(dList)/numPoints for dList in unzipped]

#         return Point(centroid_coords)
        
    


In [177]:
# def kmeans(points, k, cutoff):

#     # Pick out k random points to use as our initial centroids
#     initial = random.sample(points, k)

#     # Create k clusters using those centroids
#     # Note: Cluster takes lists, so we wrap each point in a list here.
#     clusters = [Cluster([p]) for p in initial]

#     # Loop through the dataset until the clusters stabilize
#     loopCounter = 0
#     while True:
#         # Create a list of lists to hold the points in each cluster
#         lists = [[] for _ in clusters]
#         clusterCount = len(clusters)

#         # Start counting loops
#         loopCounter += 1
#         # For every point in the dataset ...
#         for p in points:
#             # Get the distance between that point and the centroid of the first
#             # cluster.
#             smallest_distance = getDistance(p, clusters[0].centroid)

#             # Set the cluster this point belongs to
#             clusterIndex = 0

#             # For the remainder of the clusters ...
#             for i in range(clusterCount - 1):
#                 # calculate the distance of that point to each other cluster's
#                 # centroid.
#                 distance = getDistance(p, clusters[i+1].centroid)
#                 # If it's closer to that cluster's centroid update what we
#                 # think the smallest distance is
#                 if distance < smallest_distance:
#                     smallest_distance = distance
#                     clusterIndex = i+1
#             # After finding the cluster the smallest distance away
#             # set the point to belong to that cluster
#             lists[clusterIndex].append(p)

#         # Set our biggest_shift to zero for this iteration
#         biggest_shift = 0.0

#         # For each cluster ...
#         for i in range(clusterCount):
#             # Calculate how far the centroid moved in this iteration
#             shift = clusters[i].update(lists[i])
#             # Keep track of the largest move from all cluster centroid updates
#             biggest_shift = max(biggest_shift, shift)

#         # If the centroids have stopped moving much, say we're done!
#         if biggest_shift < cutoff:
#             print("Converged after %s iterations" % str(loopCounter))
#             break
#     return clusters

# def getDistance(a, b):
#     '''
#     Euclidean distance between two n-dimensional points.
#     https://en.wikipedia.org/wiki/Euclidean_distance#n_dimensions
#     Note: This can be very slow and does not scale well
#     '''
#     if a.n != b.n:
#         raise Exception("ERROR: non comparable points")

#     accumulatedDifference = 0.0
#     for i in range(a.n):
#         squareDifference = pow((a.coords[i]-b.coords[i]), 2)
#         accumulatedDifference += squareDifference
#     distance = math.sqrt(accumulatedDifference)

#     return distance

# def makeRandomPoint(n, lower, upper):
#     '''
#     Returns a Point object with n dimensions and values between lower and
#     upper in each of those dimensions
#     '''
#     p = Point([random.uniform(lower, upper) for _ in range(n)])
#     return p

# def plotClusters(data, dimensions):
#     '''
#     This uses the plotly offline mode to create a local HTML file.
#     This should open your default web browser.
#     '''
#     if dimensions not in [2, 3]:
#         raise Exception("Plots are only available for 2 and 3 dimensional data")

#     # Convert data into plotly format.
#     traceList = []
#     for i, c in enumerate(data):
#         # Get a list of x,y coordinates for the points in this cluster.
#         cluster_data = []
#         for point in c.points:
#             cluster_data.append(point.coords)

#         trace = {}
#         centroid = {}
#         if dimensions == 2:
#             # Convert our list of x,y's into an x list and a y list.
#             trace['x'], trace['y'] = zip(*cluster_data)
#             trace['mode'] = 'markers'
#             trace['marker'] = {}
#             trace['marker']['symbol'] = i
#             trace['marker']['size'] = 12
#             trace['name'] = "Cluster " + str(i)
#             traceList.append(Scatter(**trace))
#             # Centroid (A trace of length 1)
#             centroid['x'] = [c.centroid.coords[0]]
#             centroid['y'] = [c.centroid.coords[1]]
#             centroid['mode'] = 'markers'
#             centroid['marker'] = {}
#             centroid['marker']['symbol'] = i
#             centroid['marker']['color'] = 'rgb(200,10,10)'
#             centroid['name'] = "Centroid " + str(i)
#             traceList.append(Scatter(**centroid))
#         else:
#             symbols = [
#                 "circle",
#                 "square",
#                 "diamond",
#                 "circle-open",
#                 "square-open",
#                 "diamond-open",
#                 "cross", "x"
#             ]
#             symbol_count = len(symbols)
#             if i > symbol_count:
#                 print("Warning: Not enough marker symbols to go around")
#             # Convert our list of x,y,z's separate lists.
#             trace['x'], trace['y'], trace['z'] = zip(*cluster_data)
#             trace['mode'] = 'markers'
#             trace['marker'] = {}
#             trace['marker']['symbol'] = symbols[i]
#             trace['marker']['size'] = 12
#             trace['name'] = "Cluster " + str(i)
#             traceList.append(Scatter3d(**trace))
#             # Centroid (A trace of length 1)
#             centroid['x'] = [c.centroid.coords[0]]
#             centroid['y'] = [c.centroid.coords[1]]
#             centroid['z'] = [c.centroid.coords[2]]
#             centroid['mode'] = 'markers'
#             centroid['marker'] = {}
#             centroid['marker']['symbol'] = symbols[i]
#             centroid['marker']['color'] = 'rgb(200,10,10)'
#             centroid['name'] = "Centroid " + str(i)
#             traceList.append(Scatter3d(**centroid))

#     title = "K-means clustering with %s clusters" % str(len(data))
#     plotly.offline.plot({
#         "data": traceList,
#         "layout": Layout(title=title)
#     })


In [2]:
# plotly = False
# try:
#     import plotly
#     from plotly.graph_objs import Scatter, Scatter3d, Layout
# except ImportError:
#     print("INFO: Plotly is not installed, plots will not be generated.")

# def main():

#     # How many points are in our dataset?
#     num_points = 60000

#     # For each of those points how many dimensions do they have?
#     # Note: Plotting will only work in two or three dimensions
#     dimensions = (28,28)

#     # Bounds for the values of those points in each dimension
#     lower = 0
#     upper = 60000

#     # The K in k-means. How many clusters do we assume exist?
#     num_clusters = 5

#     # When do we say the optimization has 'converged' and stop updating clusters
#     cutoff = 0.2

#     # Generate some points to cluster
# #     points = [
# #         makeRandomPoint(dimensions, lower, upper) for i in xrange(num_points)
# #     ]
#     dict_mnist_data = sp.io.loadmat("mnist_data/images.mat")
#     points = dict_mnist_data['images'].T

#     # Cluster those data!
#     clusters = kmeans(points, num_clusters, cutoff)

#     # Print our clusters
#     for i, c in enumerate(clusters):
#         for p in c.points:
#             print(" Cluster: " + str(i) + ", \t Point :" + str(p))

#     # Display clusters using plotly for 2d data
#     if dimensions in [2, 3] and plotly:
#         print("Plotting points, launching browser ...")
#         plotClusters(clusters, dimensions)

In [1]:
# main()

In [136]:
''' borrowed from http://flothesof.github.io/k-means-numpy.html '''



# def move_centroids(points, closest, centroids):
#     """returns the new centroids assigned from the points closest to them"""
#     return np.array([points[closest==k].mean(axis=0) for k in range(centroids.shape[0])])

In [4]:
# points.T.shape

In [3]:
# plt.subplot(121)
# plt.scatter(points.T[:, 0], points.T[:, 1])
# centroids = initialize_centroids(points.T, 3)
# plt.scatter(centroids[:, 0], centroids[:, 1], c='r', s=100)

# plt.subplot(122)
# plt.scatter(points.T[:, 0], points.T[:, 1])
# closest = closest_centroid(points.T, centroids)
# centroids = move_centroids(points.T, closest, centroids)
# plt.scatter(centroids[:, 0], centroids[:, 1], c='r', s=100)

In [149]:
# plt.scatter(points[:, 0], points[:, 1])
# centroids = initialize_means(points, 5)
# plt.scatter(centroids[:, 0], centroids[:, 1], c='r', s=100)

(3, 28)

In [153]:
closest.shape

(60000, 28)

In [99]:

def kMeans(k, points, numIterations):
    '''
    Alternate between
        (1) yj’s are fixed; update μi’s
        (2) μi’s are fixed; update yj’s
        Halt when step (2) changes no assignments.

    Forgy method: choose k random sample points to be initial μi’s; go to (2).
    '''
    numPoints = points.shape[0]
    clusters = [[] for _ in range(k)]
    centroids = initialize_centroids(points, k)
    centroid_matrix = np.vstack(c for c in centroids)
    while numIterations:
        #step 1: fix means, change points
        for index in range(numPoints):
            distances = []
            point = points[index]
            for mean in centroids:
                diff = np.linalg.norm(point-mean)
                distances.append(diff)
            cluster = np.argmin(distances)
            clusters[cluster].append(index)   
        #step 2: fix points, change means
        centroids = []
        for c in clusters:
            cluster_points = points[c]
            new_center = np.mean(cluster_points, axis=0)
            centroids.append(new_center)
        print('iter ' + str(numIterations))
        numIterations -=1
    return clusters, centroids

def visualizeCentroids(centroids):
    plt.plot(centroids)
    plt.show()

def visualizeClusters(clusters):
    plt.plot(clusters)
    

def initialize_centroids(points, k):
    """returns k centroids and their corresponding indices from the initial points"""
    numSamples = points.shape[0]
    rand_k_indices = np.random.randint(0, numSamples, size=k).tolist()
    rand_k_means = points[rand_k_indices]
    return rand_k_means

In [100]:
k = [5, 10, 20]
np.random.shuffle(points)
points = np.reshape(points, (784, 60000))
clusters, centroids = kMeans(k[0], points, 100)

iter 100
iter 99
iter 98
iter 97
iter 96
iter 95
iter 94
iter 93
iter 92
iter 91
iter 90
iter 89
iter 88
iter 87
iter 86
iter 85
iter 84
iter 83
iter 82
iter 81
iter 80
iter 79
iter 78
iter 77
iter 76
iter 75
iter 74
iter 73
iter 72
iter 71
iter 70
iter 69
iter 68
iter 67
iter 66
iter 65
iter 64
iter 63
iter 62
iter 61
iter 60
iter 59
iter 58
iter 57
iter 56
iter 55
iter 54
iter 53
iter 52
iter 51
iter 50
iter 49
iter 48
iter 47
iter 46
iter 45
iter 44
iter 43
iter 42
iter 41
iter 40
iter 39
iter 38
iter 37
iter 36
iter 35
iter 34
iter 33
iter 32
iter 31
iter 30
iter 29
iter 28
iter 27


KeyboardInterrupt: 